In [10]:
#!/usr/bin/env python3
from subprocess import run
from random import randint
from hashlib import shake_128
from sys import argv, exit

def main(binary: str, hash_count: int):
    absent = 0
    present = 0
    for i in range(hash_count):
        hash = shake_128(randint(0, 2**36).to_bytes(5, 'little')).hexdigest(5)[:9]
        result = run(f'~/llvm-abom/llvm-abom-check {binary} {hash}', capture_output=True, shell=True)
        if result.stdout.startswith(b'Present'):
            print(f'Present: {hash}')
            present += 1
        elif result.stdout.startswith(b'Absent'):
            absent += 1
        else:
            print(f'Error: {hash}')
            exit(1)
    if (present):
        print('===\n')
    print(f'Absent: {absent}\nPresent: {present}\nError Rate: {present / hash_count:.0%}')

if __name__ == '__main__':
    if len(argv) != 3:
        print(f'Usage: {argv[0]} <binary> <hash_count>')
    else:
        main(argv[1], int(argv[2]))

Usage: /Users/nick/.virtualenvs/abom/lib/python3.11/site-packages/ipykernel_launcher.py <binary> <hash_count>


In [11]:
main(1)

TypeError: main() missing 1 required positional argument: 'hash_count'

In [15]:
from random import choices
from string import ascii_lowercase
from os import makedirs, exit
from os.path import join

def main(directory: str, count: int):
    try:
        makedirs(join(directory, 'src'))
        makedirs(join(directory, 'include'))
        makedirs(join(directory, 'bin'))
    except FileExistsError:
        print('Directory already exists')
        exit(1)
    deps = set()
    for i in range(count):
        while True:
            name = ''.join(choices(ascii_lowercase, k=8))
            if name not in deps:
                deps += name
                break
        with open(join(directory, f'src/{name}.c'), 'w') as f:
            f.write(f'void {name}() {{}}')
        with open(join(directory, f'include/{name}.h'), 'w') as f:
            f.write(f'#ifndef {name.upper()}_H\n#define {name.upper()}_H\nvoid {name}();\n#endif')
    with open(join(directory, f'src/abom{i}.c'), 'w') as f:
        for dep in deps:
            f.write(f'#include "{dep}.h"\n')
        f.write('\nint main() {\n   return 0;\n}\n')

In [16]:
main('test', 2)

FileExistsError: [Errno 17] File exists: 'test/src'